In [ ]:
import google.generativeai as genai

# Configure API Key
genai.configure(api_key="")

In [ ]:
def generate_bengali_summary_v2(content):
    prompt = f"""
    User:
    Act as an expert in Bengali Summary Generation.
    You are given some language descriptions in Bengali of a first person view video. The video is 3 minute long.
    Each sentence describes a 1s clip. Here are the descriptions: "{content}"
    Please generate a 500 words summary that captures the key details of the descriptions while preserving the most
    relevant aspects of the content.

    Assistant:
    """

    # Call Gemini API
    generation_config = {
        "temperature": 0,
    }
    response = genai.GenerativeModel("gemini-2.0-flash", generation_config=generation_config).generate_content(prompt)
    return response.text if response.parts else None

In [ ]:
# Define Bengali MCQ Prompt with User-Assistant Format
def generate_bengali_mcq_answer_gemini_v3(summary, question, options):
    prompt = f"""
    User:
    Act as an expert in Bengali MCQ Answer Generation.
    You are given some language descriptions in Bengali of a first person view video. The video is 3 minutes long.
    Each sentence describes a 1s clip. Here are the descriptions: "{summary}"
    You are going to answer a multiple choice question based on the descriptions. Please provide a single-letter answer
    to the following multiple-choice question, and your answer must be one of the letters (0, 1, 2, 3, or 4).
    You must not provide any other response or explanation.
    Here is the question: "{question}"
    Here are the choices. 0: "{options[0]}". 1: "{options[1]}". 2: "{options[2]}". 3: "{options[3]}". 4: "{options[4]}".

    Assistant:
    """

    # Call Gemini API
    generation_config = {
        "temperature": 0,
    }
    response = genai.GenerativeModel("gemini-2.0-flash", generation_config=generation_config).generate_content(prompt)
    return response.text if response.parts else None

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Long Range Video Question Answering/Original Dataset/Questions_Answer.csv")
df.head()

In [ ]:
import time
pred = []
time_taken = []
for _, row in df.iterrows():
  start = time.process_time()
  content = row['bengali_text']
  question = row['question']
  options = [(row['option 0'], row['option 1'], row['option 2'], row['option 3'], row['option 4'])]
  summary = generate_bengali_summary_v2(content)
  time.sleep(30)
  pred.append(generate_bengali_mcq_answer_gemini_v3(summary, question, options[0]))
  time_ = time.process_time() - start
  print("Response time :", time_)
  time_taken.append(time_)
  time.sleep(30)

In [ ]:
pred_cl = [p.split("\n")[0] for p in pred]
truth = df[['truth']]

In [ ]:
sum(1 for x,y in zip(truth,pred_cl) if x == y) / len(truth)

In [ ]:
sum(time_taken)/26